In [5]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [8]:
# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [10]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [12]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [13]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [14]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [15]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
3010/3010 [==============================] - 95s 31ms/step - loss: 3.7361 - accuracy: 0.2721
Epoch 2/10
3010/3010 [==============================] - 98s 33ms/step - loss: 3.3431 - accuracy: 0.3108
Epoch 3/10
3010/3010 [==============================] - 95s 31ms/step - loss: 3.0568 - accuracy: 0.3522
Epoch 4/10
3010/3010 [==============================] - 96s 32ms/step - loss: 2.8198 - accuracy: 0.3920
Epoch 5/10
3010/3010 [==============================] - 99s 33ms/step - loss: 2.6189 - accuracy: 0.4295
Epoch 6/10
3010/3010 [==============================] - 149s 49ms/step - loss: 2.4381 - accuracy: 0.4629
Epoch 7/10
3010/3010 [==============================] - 107s 35ms/step - loss: 2.2772 - accuracy: 0.4963
Epoch 8/10
3010/3010 [==============================] - 140s 47ms/step - loss: 2.1335 - accuracy: 0.5254
Epoch 9/10
3010/3010 [==============================] - 119s 40ms/step - loss: 2.0009 - accuracy: 0.5533
Epoch 10/10
3010/3010 [==============================] - 118

In [19]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 23ms/step
I will leave if they will go back
